In [32]:
# Requirements:
# !pip install rtdl
# !pip install libzero==0.0.4

In [33]:
from typing import Any, Dict

import numpy as np
import rtdl
import scipy.special
import sklearn.datasets
import sklearn.metrics
import sklearn.model_selection
import sklearn.preprocessing
import torch
import torch.nn as nn
import torch.nn.functional as F
import zero
import pandas as pd

In [34]:
device = torch.device('cpu')
# Docs: https://yura52.github.io/zero/0.0.4/reference/api/zero.improve_reproducibility.html
zero.improve_reproducibility(seed=123456)

AttributeError: module 'zero' has no attribute 'improve_reproducibility'

### Data

In [35]:
from sklearn.datasets import fetch_kddcup99 
data = fetch_kddcup99(subset='train')

from pprint import pprint
print(list(data.target_names))

['labels']


In [36]:
path = "data/Forest_Cover/train_centered.csv"

target = "Cover_Type"

In [37]:
df = pd.read_csv(path)
df["target"] = df.Cover_Type# df.income
df = df.drop(target, axis=1)

In [38]:
strCol = []
for column in df.columns:
    print(column)
    if type(df[column][0]) is str:
        strCol.append(column)


Id
Elevation
Aspect
Slope
Distance_To_Hydrology
Horizontal_Distance_To_Hydrology
Vertical_Distance_To_Hydrology
Horizontal_Distance_To_Roadways
Hillshade_9am
Hillshade_Noon
Hillshade_3pm
Horizontal_Distance_To_Fire_Points
Wilderness_Area1
Wilderness_Area2
Wilderness_Area3
Wilderness_Area4
Soil_Type1
Soil_Type2
Soil_Type3
Soil_Type4
Soil_Type5
Soil_Type6
Soil_Type7
Soil_Type8
Soil_Type9
Soil_Type10
Soil_Type11
Soil_Type12
Soil_Type13
Soil_Type14
Soil_Type15
Soil_Type16
Soil_Type17
Soil_Type18
Soil_Type19
Soil_Type20
Soil_Type21
Soil_Type22
Soil_Type23
Soil_Type24
Soil_Type25
Soil_Type26
Soil_Type27
Soil_Type28
Soil_Type29
Soil_Type30
Soil_Type31
Soil_Type32
Soil_Type33
Soil_Type34
Soil_Type35
Soil_Type36
Soil_Type37
Soil_Type38
Soil_Type39
Soil_Type40
Wilderness_Area
Area
Soil_Type
Type
Train
target


In [39]:
strCol = ["Soil_Type", "Wilderness_Area"]

In [40]:
# df = pd.DataFrame(data.data.tolist(), columns=data.feature_names)
oldNames = df.columns# data.feature_names

In [41]:
# df["target"] = data.target
# df["target"] = df.income

In [42]:
output = df.target.values
labels = set(output)
print('The different type of output labels are:',labels)
print('='*125)
print('No. of different output labels are:', len(labels))

The different type of output labels are: {1, 2, 3, 4, 5, 6, 7}
No. of different output labels are: 7


In [43]:
def onhotencode(_df, _col):
    _values = set(_df[_col].values)
    for v in _values:
        _df[v] = _df[_col].apply(lambda x : float(x == v) )
    return _df

In [44]:
for c in df.columns:
    if (not c in strCol) and (c != "target"):
        df = df.drop(c, axis=1)

In [45]:
for col in df.columns:
    if col != "target":
        df = onhotencode(df, col)
        df = df.drop(col, axis=1)

In [46]:
df["target"] = df["target"].apply(lambda x: str(x))

In [30]:
df.to_csv("data/Forest_Cover/training_processed.csv")

In [47]:
df

,target,1,2,3,4,5,6,8,9,10,...,31,32,33,34,35,36,37,38,39,40
0,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15115,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15116,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15117,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15118,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
list(df.columns)

['target',
 'OTHER',
 'ADESA',
 'MANHEIM',
 'DODGE',
 'SATURN',
 'JEEP',
 'CADILLAC',
 'HONDA',
 'SCION',
 'HYUNDAI',
 'CHEVROLET',
 'MERCURY',
 'NISSAN',
 'VOLKSWAGEN',
 'ISUZU',
 'INFINITI',
 'KIA',
 'MINI',
 'BUICK',
 'TOYOTA',
 'SUBARU',
 'LINCOLN',
 'PLYMOUTH',
 'HUMMER',
 'SUZUKI',
 'FORD',
 'CHRYSLER',
 'TOYOTA SCION',
 'PONTIAC',
 'MITSUBISHI',
 'OLDSMOBILE',
 'MAZDA',
 'GMC',
 'LEXUS',
 'ACURA',
 'VOLVO',
 'LEGACY',
 'SILHOUETTE 3.4L V6 S',
 'TAURUS 3.0L V6 EFI D',
 'SX4 2.0L I4 EFI',
 '1500HD SILVERADO PIC',
 'XG 350',
 'CAMRY V6 3.5L V6 SFI',
 'GALANT 4C 2.4L I4 MP',
 'FOCUS',
 'COROLLA 1.8L I-4 EFI',
 'COOPER 1.6L I-4 EFI',
 'RENDEZVOUS FWD 3.4L',
 'MURANO 2WD V6',
 'TAHOE 4WD 4.8L V8 SF',
 'ASCENDER 2WD 6C',
 'TUNDRA PICKUP 2WD V6',
 'TRACKER 2WD 4C',
 'GRAND AM 4C 2.4L I-4',
 'XTERRA 2WD V6 4.0L V',
 'SABLE',
 'COBALT 2.2L I4 MPI',
 'CENTURY V6 3.1L V6 S',
 'LIBERTY 4WD V6',
 'STRATUS V6 2.7L V6 S',
 'TACOMA PICKUP 2WD V6',
 'HIGHLANDER 4WD 4C',
 'TSX',
 'SEQUOIA 2WD',
 '